In [1]:
import pkgutil
import json

from openfisca_france import model
from openfisca_france.model.base import Variable

In [2]:
# Import all modules from OpenFisca-France model (so that __subclass__ finds them)
__all__ = []
for loader, module_name, is_pkg in pkgutil.walk_packages(model.__path__):
    __all__.append(module_name)
    _module = loader.find_module(module_name).load_module(module_name)
    globals()[module_name] = _module

In [3]:
variables = []
for cls in Variable.__subclasses__():
    try:
        cerfa = cls.cerfa_field
    except AttributeError:
        continue
    if isinstance(cerfa, dict):
        boxes = [cerfa[k] for k in sorted(cerfa)]
    elif isinstance(cerfa, str):
        boxes = [cerfa]
    else:
        raise ValueError("cerfa_field is neither a dict nor a string.")
    variable = {
        'boxes': boxes,
        'type': cls.value_type.__name__,
        'description': getattr(cls, 'label', None),
    }
    variables.append(variable)

In [4]:
variables += [
    {
        'boxes': ["8HV", "8IV", "8JV", "8KV"],
        'type': 'float',
        'description': "Prélèvement à la source déjà payé - retenue à la source sur les salaires et pensions",
    },
    {
        'boxes': ["8HW", "8IW", "8JW", "8KW"],
        'type': 'float',
        'description': "Prélèvement à la source déjà payé - acomptes d'impôt sur le revenu",
    },
    {
        'boxes': ["8HX", "8IX", "8JX", "8KX"],
        'type': 'float',
        'description': "Prélèvement à la source déjà payé - acomptes de prélèvements sociaux",
    },
    {
        'boxes': ["8HY", "8IY", "8JY", "8KY"],
        'type': 'float',
        'description': "Prélèvement à la source - Remboursement de trop-prélevé déjà obtenu - impôt sur le revenu",
    },
    {
        'boxes': ["8HZ", "8IZ", "8JZ", "8KZ"],
        'type': 'float',
        'description': "Prélèvement à la source déjà payé - Remboursement de trop-prélevé déjà obtenu - prélèvements sociaux",
    },
]

In [7]:
with open('cerfa_variables.json', 'w+') as f:
    json.dump(variables, f, indent=2)